In [1]:
import math, random
import string

class Cromossomo():

    def __init__(self, tamanho):
        self.lista_modelos = self.criaListaModelos()
        self.tamanho = tamanho
        self.valor = []
        self.avaliacao = 0

    
    def criaListaModelos(self):##Esta sendo usado
        mod1 = list(range(0,10))
        mod2 = list(string.ascii_uppercase)
        
        #modelo = [[1..9],[A..Z]]
        modelo = [mod1, mod2]
        
        return modelo
        
    def getLetraRandom(self):##Esta sendo usado
        i = random.randint(0,25)
        
        return self.lista_modelos[1][i]
    
    def set_valor(self, novo_valor):##Esta sendo usado
        self.valor = novo_valor
    
    def inicializar(self):##Esta sendo usado
        novo_valor = []
        for i in range(self.tamanho):
            if random.random() < .5: ##Se valor.randômico < 0.50, adiciona um numero aleatório
                novo_valor.append(random.randint(0,9))
            else: novo_valor.append(self.getLetraRandom())
                
        self.set_valor(novo_valor)

    def crossover(self, outro_cromossomo):
        split_index = int(random.random() * self.tamanho) ##Gera o índice onde vai ocorrer a troca de Genes
        novo_valor = []
        if random.random() > .5:##Se a probabilidade for maior que 50%
            novo_valor = self.valor[0:split_index] + outro_cromossomo.valor[split_index:len(outro_cromossomo.valor)]
        else:
            novo_valor = outro_cromossomo.valor[0:split_index] + self.valor[split_index:len(outro_cromossomo.valor)]
        novo_cromossomo = Cromossomo(self.tamanho)
        novo_cromossomo.set_valor(novo_valor)
        return novo_cromossomo

    def mutacao(self, chance_mutacao):
        inicio, fim = [[],[]]
        for i in range(self.tamanho):
            if random.random() < chance_mutacao:
                inicio = self.valor[0:i]
                fim = self.valor[i+1:self.tamanho]
                aux = self.valor[i]
                if type(aux) == int : aux = self.getLetraRandom() ##Coloca uma letra aleatória na posição mutada
                else: aux = random.randint(0,9) ##Coloca um número, caso contrário
                self.valor = self.concatena_valores(inicio,aux,fim)
                
    def concatena_valores(self, a ,b ,c):
        resultado = []
        for i in range(len(a)):
            resultado.append(a[i])
            
        resultado.append(b)
        
        for i in range(len(c)):
            resultado.append(c[i])
        
        return resultado
            

    def valor_real(self, inf = 0, sup = 100):
        return inf + (sup - inf)/(2**self.tamanho - 1)*int(self.valor, 2)

    def avaliar(self, modelo):##Esta sendo usado
        for i in range(len(modelo)):
            if type(self.valor[i]) == type(modelo[i]):
                self.avaliacao += 1

        return self.avaliacao

    def __repr__(self):

        return "cromossomo:[%s] avaliacao[%.2f]" % (self.valor, self.avaliacao)

In [6]:
import math, random
import string
import re

class GATest():

    def __init__(self, tamanho_populacao, geracoes):
        self.populacao = []
        self.tam_populacao = tamanho_populacao
        self.geracoes = geracoes
        self.soma_avaliacoes = 0
        self.melhores = []
    
    
    def inicializar_populacao(self, tam_cromossomo):
        for i in range(self.tam_populacao):
            self.populacao.append(Cromossomo(tam_cromossomo))
        for cromossomo in self.populacao: cromossomo.inicializar()
            
    def avaliar_populacao(self,modelo):
        for cromossomo in self.populacao:
            self.soma_avaliacoes += cromossomo.avaliar(modelo)
        
        return sorted(self.populacao, key = lambda x: x.avaliacao, reverse=True)

    def roleta(self):
        limite = random.random() * self.soma_avaliacoes ##Gera um limite baseado na avaliação da População
        i, aux = [0, 0]
        ##random.shuffle(self.populacao) ## "Embaralha" a população aleatoriamente
        while aux < limite and i < self.tam_populacao:
            aux += self.populacao[i].avaliacao
            i += 1
        i -= 1
        return i       

    def nova_geracao(self):
        nova_populacao = []
        for i in range(self.tam_populacao):
            pai1 = self.populacao[self.roleta()]
            pai2 = self.populacao[self.roleta()]
            filho = pai1.crossover(pai2)
            filho.mutacao(.05)
            nova_populacao.append(filho)
        return nova_populacao
    
    def informaModelo(self):
        print("Digite seu modelo:")
        input_model = input()
        input_model_split = input_model.split()
        model = []
        for i in input_model_split:
            if re.match('^\d+$',i): model.append(int(i))
            else: model.append(i)
                
        return model
    
    def executar(self):
        model = self.informaModelo()
        populacoes = []
        tam_model = len(model)
        
        self.inicializar_populacao(tam_model)
        ordenado = self.avaliar_populacao(model)
        
        
        for i in range(self.geracoes):
            self.melhores.append(ordenado[0])
            populacoes.append(self.populacao)
            for top in ordenado:
                print("geração {}, Cromossomo:{}".format(i, top))
            
            self.populacao = self.nova_geracao()
            ordenado = self.avaliar_populacao(model)
        
        print("\n############## BESTS ######################\n")
        for top in sorted(self.melhores, key = lambda x: x.avaliacao, reverse = True):
            print(top)
        print("\n############## BESTS ######################\n")

ga = GATest(5,20)
ga.executar()

Digite seu modelo:
A 2 3 G 1 T
geração 0, Cromossomo:cromossomo:[['R', 2, 3, 2, 4, 'R']] avaliacao[5.00]
geração 0, Cromossomo:cromossomo:[['T', 8, 'O', 'F', 1, 'O']] avaliacao[5.00]
geração 0, Cromossomo:cromossomo:[['E', 5, 'Y', 9, 'O', 'N']] avaliacao[3.00]
geração 0, Cromossomo:cromossomo:[[1, 'Y', 2, 1, 8, 2]] avaliacao[2.00]
geração 0, Cromossomo:cromossomo:[['E', 'X', 'R', 6, 'N', 4]] avaliacao[1.00]
geração 1, Cromossomo:cromossomo:[['R', 2, 3, 'F', 1, 'O']] avaliacao[6.00]
geração 1, Cromossomo:cromossomo:[['T', 8, 'O', 'F', 1, 'R']] avaliacao[5.00]
geração 1, Cromossomo:cromossomo:[['E', 5, 'Y', 2, 4, 'R']] avaliacao[4.00]
geração 1, Cromossomo:cromossomo:[['T', 'P', 'O', 'F', 1, 'O']] avaliacao[4.00]
geração 1, Cromossomo:cromossomo:[['E', 'X', 'R', 6, 4, 'R']] avaliacao[3.00]
geração 2, Cromossomo:cromossomo:[['R', 2, 3, 'F', 1, 'R']] avaliacao[6.00]
geração 2, Cromossomo:cromossomo:[['R', 2, 'O', 'F', 1, 'R']] avaliacao[5.00]
geração 2, Cromossomo:cromossomo:[['E', 5, 6, 6